In [1]:
import urllib3
import requests
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, MultiLineString
import shapely
import tqdm
import copy

In [6]:
def submit_http_api_request(bbox_str, dataset, url):
    payload = {'bbox': bbox_str, "datasets": dataset}
    response = requests.get(url, payload)
    #print(response.url)
    # We can check whether the request is sucessful, (200 -> OK) or nother errors (i.e, 404 -> Not Found)
    if response.status_code  == 200:
        response.encoding = 'utf-8' # Optional: requests infers this internally
        contents_json = response.json()
        return(True, contents_json)
    else:         
        print("An error occurred", response.status_code)
        return(False, _)

def verify_entries_exist(contents_json, entries_idx):
    if int(contents_json[entries_idx]) == 0:
        return(False)
    else:
        return(True)

def json_to_dict(json, json_idx, nested_idx):
    dict_ = {}
    for item in json[json_idx]:
        col_idx = item.keys()
        title = item[nested_idx]
        dict_[title] = item
    df = pd.DataFrame(dict_,index=col_idx)
    df = df.T
    return(df)

#get attributes of lpc
def get_dataset_of_interest(contents_df, name_list, url_list, geometry_list):
    newest = contents_df.loc[contents_df["dateCreated"] == contents_df["dateCreated"].max()]
    url = newest['downloadLazURL'][0]
    bbox = newest['boundingBox'][0]
    polygon = Polygon([(bbox["minX"],bbox["minY"]), (bbox["minX"],bbox["maxY"]), 
                         (bbox["maxX"],bbox["maxY"]), (bbox["maxX"],bbox["minY"])])
    #add to list
    name_list.append(newest.index[0])
    url_list.append(url)
    geometry_list.append(polygon)
    return(name_list, url_list, geometry_list)

In [3]:
tnm_url = 'https://tnmaccess.nationalmap.gov/api/v1/products'

# read tile level AST data

In [4]:
tile_level_annotations_path = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset_/tile_level_annotations/tile_level_annotations.geojson"
tile_level_annotations = gpd.read_file(tile_level_annotations_path)
#tile_level_annotations.columns

In [ ]:
confirm that the total number of enteries is greater than 1


In [ ]:
tnm_names = []
urls = []
geometries = []
tank_idxs = []

for tank_idx, row in tile_level_annotations.iterrows():
    minx = row['nw_corner_polygon_lon']
    miny = row['se_corner_polygon_lat']
    maxx = row['se_corner_polygon_lon']
    maxy = row['nw_corner_polygon_lat']
    tank_polygon = Polygon([(minx,miny), (minx,maxy), (maxx,maxy), (maxx,miny)])
    tank_bbox = [minx, miny, maxx, maxy]
    tank_bbox_str = ",".join(map(str, tank_bbox)) #https://stackoverflow.com/questions/32313623/passing-comma-separated-values-in-request-get-python
    #print(tank_idx, tank_bbox_str)
    
    tnm_names_itr = copy.copy(tnm_names)
    urls_itr = copy.copy(urls)
    geometries_itr = copy.copy(geometries)

    #check if tank polygon is inside of a previously identified dataset
    #for i, (name, url, tnm_poly) in enumerate(zip(tnm_names_copy, urls_copy, geometries_copy)):#iterate over the tank polygons
    #    if not tnm_poly.contains(tank_polygon): matching_poly 
    matching_poly= [(tnm_name, url, tnm_poly) for tnm_name, url, tnm_poly in zip(tnm_names_itr, urls_itr, geometries_itr) if tnm_poly.contains(tank_polygon)]
    
    if len(matching_poly) > 0:
        #print(tank_idx, "already found lpc data", len(matching_poly))
        tnm_name, url, tnm_poly = matching_poly[0]
        tnm_names.append(tnm_name)
        urls.append(url)
        geometries.append(tnm_poly)
        tank_idxs.append(tank_idx)
    else:
        bool_request, lpc_contents_json = submit_http_api_request(tank_bbox_str, "Lidar Point Cloud (LPC)", tnm_url)
        if bool_request:
        #if bool_request == True:
            bool_entries = verify_entries_exist(lpc_contents_json, "total")
            if bool_entries:
                lpc_contents_df = json_to_dict(lpc_contents_json, "items", "title") 
                tnm_names, urls, geometries = get_dataset_of_interest(lpc_contents_df, tnm_names, urls, geometries)
                tank_idxs.append(tank_idx)
        #print(tank_idx, "identified, lpc poly")


In [71]:
lpc_contents_json

{'total': 0,
 'items': [],
 'errors': [],
 'messages': ['Retrieved 0 item(s) '],
 'sciencebaseQuery': 'https://www.sciencebase.gov/catalog/items?filter=tagslq%3Dtags.scheme%3A%22The+National+Map+Collection+Thesaurus%22+AND+%28tags.name%3A+%22Lidar+Point+Cloud+%28LPC%29%22%29&filter=spatialQuery%3D%7B%22wkt%22%3A%22POLYGON%28%28-119.73968635595665+35.47991829680595%2C-119.73921105688328+35.47991829680595%2C-119.73921105688328+35.48028605968282%2C-119.73968635595665+35.48028605968282%2C-119.73968635595665+35.47991829680595%29%29%22%2C%22relation%22%3A%22intersects%22%7D&max=50&offset=0&fields=id%2Ctitle%2Csummary%2Cbody%2Ctags%2CwebLinks%2Cdates%2Cspatial%2Cidentifiers&format=json&sort=title&order=asc',
 'filteredOut': 0}

In [58]:
matching_poly

[]

In [51]:
[(name, url, tnm_poly) for name, url, tnm_poly in zip(names_itr, urls_itr, geometries_itr) if tnm_poly.contains(tank_polygon)]

[('USGS Lidar Point Cloud NY LongIsland-Z18 2014 18TXK060970 LAS 2015',
  'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/USGS_LPC_NY_LongIsland_Z18_2014_LAS_2015/laz/USGS_LPC_NY_LongIsland_Z18_2014_18TXK060970_LAS_2015.laz',
  <shapely.geometry.polygon.Polygon at 0x13a2f71de40>)]

In [ ]:
https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension

In [48]:
#[ expression for item in list if conditional ]


[]

In [41]:
len(tile_level_annotations)

98169

In [30]:
    #identify img/xml that have been moved
    #img_paths_copy = copy.copy(img_paths)
    #xml_paths_copy = copy.copy(xml_paths)

[]

In [37]:
lpc_name_list, lpc_url_list, geometry_list

([], [], [])

# http get in python
#https://realpython.com/python-requests/
#https://www.w3schools.com/tags/ref_httpmethods.asp
#https://stackoverflow.com/questions/645312/what-is-the-quickest-way-to-http-get-in-python

In [ ]:
Notes 

# Lidar Data
select the file with the largest bounding box/that is newest
matches file type "LAS,LAZ"

## api access
#https://ers.cr.usgs.gov/profile/access 
#https://apps.nationalmap.gov/tnmaccess/

In [101]:
#for i in range(len(lpc_json_contents["items"])):
    #print(lpc_json_contents["items"][i])

#get lpc contents


In [195]:
#for geometry_list


0it [00:00, ?it/s]


In [ ]:
#check if bbox is included in list of lpc

#request lpc

In [175]:
lpc_gdf = None
if lpc_gdf == None:
    lpc_gdf = gpd.GeoDataFrame({'lpc_name': lpc_newest.index[0], "lpc_url": lpc_url, 'geometry': lpc_polygon})
    return(gdf)  


SyntaxError: incomplete input (922950664.py, line 1)

# DEM Data
select the file with the largest bounding box/that is newest
matches file type "LAS,LAZ"

In [20]:
dem_1m_dataset = ["Digital Elevation Model (DEM) 1 meter"]
dem_3m_dataset = ["National Elevation Dataset (NED) 1/9 arc-second"] #why is the list not including both
dataset_str = ",".join(map(str, dataset)) 
payload = {'bbox': bbox_str, "datasets": dataset}
response = requests.get(url, payload)
print(response.url)

https://tnmaccess.nationalmap.gov/api/v1/products?bbox=-104.92070147938492%2C39.77659825409954%2C-104.9206454910845%2C39.776652353263664&datasets=Digital+Elevation+Model+%28DEM%29+1+meter&datasets=National+Elevation+Dataset+%28NED%29+1%2F9+arc-second


In [21]:
#We can check whether the request is sucessful0
##first using the status code
status_code = response.status_code # status code, shows whether the request was sucess (200 -> OK) or not (404 -> Not Found
if status_code == 200:
    print('Success!')
elif status_code == 404:
    print('Not Found.')
    
##then use the response return itself
if response: #may not have actually worked (not just a 200 return), but returns a workable response (204 -> no content in message body for example
    print('Success!')
else:
    print('An error has occurred.')

Success!
Success!


In [22]:
#https://realpython.com/python-requests/
#get the content from  request 
request_content = response.content #payload of the get request in bytes
#print(request_content)
response.encoding = 'utf-8' # Optional: requests infers this internally
request_test = response.text #converts to a strong using a character encoder (UTF-8)
#print(request_test)
response.json()

#response.headers['Content-Type']#

{'total': 1,
 'items': [{'title': 'USGS NED ned19_n40x00_w105x00_co_denvercity_2008 1/9 arc-second 2009 15 x 15 minute IMG',
   'moreInfo': 'This tile of the National Elevation Dataset (NED) is 1/9 arc-second resolution. The National Elevation Dataset (NED) serves the elevation layer of The National Map, and provides basic elevation information for earth science studies and mapping applications in the United States. Scientists and resource managers use NED data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. The NED is an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers are composed of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. The NED is updated continually [...]',
   'sourceId': '5d0ae9c6e4b0e3d3116021b5',
   'sourceName': 'ScienceBase',
   'sourceOriginId': None,
   '

In [16]:
#https://realpython.com/python-requests/
#get the content from  request 
request_content = response.content #payload of the get request in bytes
#print(request_content)
response.encoding = 'utf-8' # Optional: requests infers this internally
request_test = response.text #converts to a strong using a character encoder (UTF-8)
#print(request_test)
response.json()
#response.headers['Content-Type']#

{'total': 2,
 'items': [{'title': 'USGS 1 Meter 13 x50y441 CO_DRCOG_2020_B20',
   'moreInfo': 'This is a tile of the standard one-meter resolution digital elevation model (DEM) produced through the 3D Elevation Program (3DEP) . The elevations in this DEM represent the topographic bare-earth surface. USGS standard one-meter DEMs are produced exclusively from high resolution light detection and ranging (lidar) source data of one-meter or higher resolution. One-meter DEM surfaces are seamless within collection projects, but, not necessarily seamless across projects. The spatial reference used for tiles of the one-meter DEM within the conterminous United States (CONUS) is Universal Transverse Mercator (UTM) in units of meters, and in conformance with the North American Datum of 1983 (NAD83). All bare earth elevation values are in [...]',
   'sourceId': '620de50ed34e6c7e83baa048',
   'sourceName': 'ScienceBase',
   'sourceOriginId': None,
   'sourceOriginName': 'gda',
   'metaUrl': 'https:/

In [ ]:
https://tnmaccess.nationalmap.gov/api/v1/products?bbox=-104.92070147938492,%20%2039.77659825409954,%20%20-104.9206454910845,%20%2039.776652353263664